# 数据生成

**AERO-S**是固体仿真模拟器，这里是详细的[**AERO-S**教程](https://bitbucket.org/frg/aero-s/downloads/AERO-S.pdf)。

本案例是使用**AERO-S**生成大量降落伞受力展开数据用于训练神经算子。分为以下几个步骤
- 参数化降落伞, 生成相关的**AERO-S**输入文件
- 使用**AERO-S**仿真计算结果

# 生成降落伞网格、材料等**AERO-S**输入文件进行仿真



## 数据信息

我们考虑预测不同降落伞在指定压力载荷下的充气过程。数据集是通过生成三种类型的降落伞来构建的：  
1. **盘-缝-带降落伞（DGB）**：一种超音速减速器，由盘、缝和带组成
2. **环帆降落伞**：具有同心、部分通风的面板（类似于“环”），用于高性能减速 
3. **带状降落伞**：具有纵向带状结构，通过加强线分隔，以增强超音速减速过程中的稳定性 

对于每种类型，通过改变参数（如伞衣幅数、直径、缝尺寸等）生成 2,000 个降落伞，具体参数详见下表。

---

### 表 1：降落伞动力学设计参数 [1]

|  $~~~$ | **设计变量** | $~~$  |
| --- | --- | --- |
|| **伞衣幅数** | $16, 20, \cdots, 36, 40$ |
|| **标称直径** | $5m \sim 9m$ |
|| **通风口直径** | $0.5m \sim 1m$ |
|| **悬挂线长度** | $10 m$ |

| **DGB 降落伞** | **伞衣高度** | $1m \sim 2m$ |
| --- | --- | --- |
| | **缝比例** | $20\%\sim30\%$ |

| **环帆降落伞** | **伞衣高度** | $1m \sim 2m$ |
| --- | --- | --- |
| | **第一环底部位置** | $25\% \sim 30\%$ |
| | **第二环顶部位置** | $35\% \sim 40\%$ |
| | **第二环底部位置** | $60\% \sim 65\%$ |
| | **第三环顶部位置** | $75\% \sim 80\%$ |

| **带状降落伞** | **伞衣高度** | $0m \sim 2m$ |
| --- | --- | --- |
| | **第一条加强线位置** | $20\% \sim 40\%$ |
| | **第二条加强线位置** | $60\% \sim 80\%$ |

**表注**：降落伞动力学：每种降落伞设计的几何参数。DGB 降落伞采用圆柱形设计，由盘、缝和带组成。环帆降落伞采用圆弧设计，由三个环结构组成，第一个环连接至通风口。这些环的顶部和底部位置通过其与通风口边缘的相对距离参数化。带状降落伞设计为锥形，包含两条嵌入的纵向加强线。悬挂线长度为固定值 $10m$，连接固定底部点与降落伞伞衣。

压力载荷在前 $0.1$ 秒内从 $0$ 线性增加到 $1000$ Pa，之后保持在 $1000$ Pa 不变。降落伞的伞衣、悬挂线和加强线材料均假设为线性弹性材料，材料属性与 [2] 中的一致。


## 脚本

我们采用自己研发的降落伞网格自动生成脚本，首先在`Parachute_Data`文件夹里用以下命令下载（需要许可）

```shell
git clone https://github.com/PKU-CMEGroup/Parachute_Design.git
```

我们的参数化函数在文件 [Parachute_Generator.py](Parachute_Design/Parachute_Generator.py) 和[Parachute_Aero_Suite.py](Parachute_Design/Parachute_Aero_Suite.py) 中。运行这些文件中的函数`generate_parachute()` 能生成固体对应的 `StructureFile.include` 文件，包括了网格以及相应的材料性质等，和"SelfContactSurfaceTopology.include"文件，包含了用于计算受力或者接触碰撞的面。随后我们能用**AERO-Suite**仿真计算结果。

对于生成数据，我们用**python**把 `Parachute` 文件夹复制为 `Parachute_XXXXX`， 提交脚本生成网格、调用**AERO-S**进行计算。这可以在 `Parachute_Data` 文件夹里用以下命令行完成

```shell
sbatch Sbatch.Generate_Data.sh 
```


# 处理计算结果

我们读入**AERO-Suite**仿真计算结果，并用 [Parachute_Postprocess.py](Parachute_Design/Parachute_Postprocess.py) 里的 `structure_data()` 函数，处理降落伞数据得到，格点坐标 (`nodes_XXXXX.npy`)，三角形、线段有限元 (`elems_XXXXX.npy`)，以及位移等 (`features_XXXXX.npy`)，把这些数据存储下来。这可以在 `Parachute_Data` 文件夹里用以下命令行完成

```shell
sbatch Sbatch.Parachute_Data.sh 
```

# 文献

1. Chenyu Zeng, Yanshu Zhang, Jiayi Zhou, Yuhan Wang, Zilin Wang, Yuhao Liu, Lei Wu, Daniel Zhengyu Huang, [Point Cloud Neural Operator for Parametric PDEs on Complex and Variable Geometries](https://arxiv.org/abs/2501.14475)


2. Daniel Zhengyu Huang, Philip Avery, Charbel Farhat, Jason Rabinovitch, Armen Derkevorkian, Lee D Peterson, [Modeling, Simulation and Validation of Supersonic Parachute Inflation Dynamics During Mars Landing](https://arc.aiaa.org/doi/abs/10.2514/6.2020-0313)